In [1]:

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch
import pandas as pd
import scanpy as sc
import numpy as np
from torch.utils.data import TensorDataset
train_data=pd.read_csv("test_data.csv")
#print(train_data)
train_data=train_data.T
train_data.to_csv("testT_data.csv")
data=train_data.values
data_values=data.T  #进行转置？
#print(train_data)
adata=sc.read_csv("testT_data.csv")#应该进行装置
adata=adata[1:201]
print(type(data))
a,gene_num=adata.shape
sc.pp.filter_cells(adata, min_counts=1)
adata.raw = adata.copy()
adata.obs['size_factors'] = adata.obs.n_counts / np.median(adata.obs.n_counts)
sc.pp.log1p(adata)
sc.pp.scale(adata)

Trying to set attribute `.obs` of view, copying.


<class 'numpy.ndarray'>


进行预处理

数据集组成为 200个细胞，每个细胞1000个基因需要进行表达

In [2]:
#编码方式
class seq_dataset(Dataset):
    def __init__(self,load_data):
        self.data = load_data
    def __getitem__(self, idx):
        return self.data[idx]
    def __len__(self,):
        return len(self.data)
RNA=seq_dataset(data_values)
train_RNA_dataloader=DataLoader(RNA,batch_size=32,shuffle=True)
count_mtx = torch.tensor(adata.X)
count_mtx_raw = torch.tensor(adata.raw.X)
size_factor = torch.tensor(adata.obs.size_factors.values)
dataset = TensorDataset(count_mtx, size_factor,count_mtx_raw)
#print(dataset)
train1_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for item in train1_dataloader:
    print(item)

问题在于：对true data的使用以及在这一过程中对dataset的封装

In [3]:
def weights_init(mat):
    if type(mat) == nn.Linear:
        torch.nn.init.xavier_uniform_(mat.weight,gain=3)
        print("Weight:",mat.weight)
        mat.bias.data.fill_(0.1)

In [4]:
class mean_activation(nn.Module):
    def __init__(self):
        super(mean_activation,self).__init__()
    def forward(self,input):
        return torch.exp(input)

class dispersion_activiation(nn.Module):
    def __init__(self):
        super(dispersion_activiation,self).__init__()
    def forward(self,input):
        return torch.exp(input)
    

In [5]:
class auto_encoder(nn.Module):
    def __init__( self,gene_num,hidden_dim):
        super(auto_encoder,self).__init__()
        self.gene_num = gene_num
        print("gene_num: " ,gene_num)

        self.autoencoder_stack=nn.Sequential(
            nn.Linear(gene_num,hidden_dim[0]),
            nn.BatchNorm1d(hidden_dim[0],eps=1e-2,affine=False), 
            nn.ReLU(),
            nn.Linear(hidden_dim[0],hidden_dim[1]),
            nn.BatchNorm1d(hidden_dim[1],eps=1e-2,affine=False),
            nn.ReLU(),
            nn.Linear(hidden_dim[1],hidden_dim[2]),
            nn.BatchNorm1d(hidden_dim[2],eps=1e-2,affine=False),
            nn.ReLU(),
        )
        self.mean=nn.Sequential(
            nn.Linear(hidden_dim[2],gene_num),
            mean_activation()
        )
        self.pi=nn.Sequential(
            nn.Linear(hidden_dim[2],gene_num),
            nn.Sigmoid()
        )
        self.theta=nn.Sequential(
            nn.Linear(hidden_dim[2],gene_num),
            dispersion_activiation()
        )

    def forward(self, input, size_factor):
        size_factor = size_factor.view(-1, 1)#?
        size_factor = size_factor.repeat(1, self.gene_num)
        #ae_out = input * size_factor
        ae_out = self.autoencoder_stack(input) #?
        #print("AE:  ", ae_out)
        mean = self.mean(ae_out) * size_factor
        #print("MEAN:  ", mean)
        theta = self.theta(ae_out)
        pi = self.pi(ae_out)
        return mean, theta, pi

        


In [6]:
def _nan2inf(x):
    return torch.where(torch.isnan(x), torch.zeros_like(x) + np.inf, x)
class NB(torch.nn.Module):
    def __init__(self, scale_factor=1.0):
        super(NB, self).__init__()
        self.eps = 1e-10
        self.scale_factor = scale_factor

    def forward(self, theta, y_true, y_pred, mean=True):
        y_pred = y_pred * self.scale_factor
        t1 = torch.lgamma(theta + self.eps) + torch.lgamma(y_true + 1.0) - torch.lgamma(y_true + theta + self.eps)
        t2 = (theta + y_true) * torch.log(1.0 + (y_pred / (theta + self.eps))) + (
                y_true * (torch.log(theta + self.eps) - torch.log(y_pred + self.eps)))

        final = t1 + t2
        final = _nan2inf(final)

        if mean:
            final = torch.mean(final)

        return final
class ZINB(NB):
    def __init__(self, scale_factor=1.0, ridge_lambda=0.0):
        super().__init__(scale_factor=scale_factor)
        self.ridge_lambda = ridge_lambda
        self.scale_factor = scale_factor

    def forward(self, pi, theta, y_true, y_pred, mean=True):
        nb_case = super().forward(theta, y_true, y_pred, mean=False) - torch.log(1.0 - pi + self.eps)

        y_pred = y_pred * self.scale_factor
        # theta = torch.minimum(theta, torch.tensor(1e6))

        zero_nb = torch.pow(theta / (theta + y_pred + self.eps), theta)
        zero_case = -torch.log(pi + ((1.0 - pi) * zero_nb) + self.eps)
        result = torch.where(torch.lt(y_true, 1e-8), zero_case, nb_case)

        if self.ridge_lambda > 0:
            ridge = self.ridge_lambda * torch.square(pi)
            result += ridge

        if mean:
            result = torch.mean(result)

        result = _nan2inf(result)

        return result

In [7]:
learning_rate = 1e-3
batch_size = 32
epochs = 180
grad_clip_val = 5.0

In [8]:

#gene_num=count_mtx_raw.shape[1]
model = auto_encoder(gene_num=gene_num, hidden_dim=(64, 32, 64))
model.apply(weights_init)#不完整

gene_num:  1000
Weight: Parameter containing:
tensor([[ 0.0156, -0.1413,  0.0533,  ...,  0.0188, -0.1021,  0.1449],
        [-0.1138, -0.1066, -0.1866,  ...,  0.1941, -0.0847,  0.1680],
        [ 0.1528, -0.1063,  0.2082,  ...,  0.2057,  0.0848, -0.1067],
        ...,
        [-0.0009, -0.1513,  0.0012,  ...,  0.1231,  0.1540,  0.1538],
        [ 0.0930, -0.1259, -0.1278,  ..., -0.1696,  0.1224,  0.2214],
        [ 0.0305,  0.1558, -0.0392,  ..., -0.1160,  0.1739,  0.2039]],
       requires_grad=True)
Weight: Parameter containing:
tensor([[-0.6372,  0.2679, -0.0640,  ..., -0.5816,  0.2378, -0.3739],
        [-0.2608, -0.1821, -0.6581,  ...,  0.1831,  0.2766, -0.4294],
        [ 0.5798, -0.4580, -0.6823,  ..., -0.1803, -0.5637, -0.3464],
        ...,
        [ 0.5452,  0.5915, -0.6691,  ...,  0.4414,  0.2958,  0.4173],
        [-0.5428,  0.2755,  0.4818,  ..., -0.4206,  0.6115, -0.5740],
        [-0.0534, -0.1294,  0.2222,  ..., -0.2887,  0.4565, -0.3400]],
       requires_grad=True)
We

auto_encoder(
  (autoencoder_stack): Sequential(
    (0): Linear(in_features=1000, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=0.01, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): BatchNorm1d(32, eps=0.01, momentum=0.1, affine=False, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=64, bias=True)
    (7): BatchNorm1d(64, eps=0.01, momentum=0.1, affine=False, track_running_stats=True)
    (8): ReLU()
  )
  (mean): Sequential(
    (0): Linear(in_features=64, out_features=1000, bias=True)
    (1): mean_activation()
  )
  (pi): Sequential(
    (0): Linear(in_features=64, out_features=1000, bias=True)
    (1): Sigmoid()
  )
  (theta): Sequential(
    (0): Linear(in_features=64, out_features=1000, bias=True)
    (1): dispersion_activiation()
  )
)

In [9]:
loss_fn = ZINB()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, epochs / 2)

weight init看着没有问题

In [10]:
def train(dataloader,model,loss,optimizer):
    #size_data=len(dataloader.dataset)
    for batch, (X, size, y) in enumerate(dataloader):
        optimizer.zero_grad()
    # Predict, and calculate loss
        mean, theta, pi = model(y, size)
        #loss_val = loss(mean, theta, pi, y)
        loss_val= loss (pi,theta,y,mean,mean=True)
        loss_val.backward()
        nn.utils.clip_grad_norm_(model.parameters(), grad_clip_val)
        optimizer.step()
        if batch % 50 == 0:
            #loss, curr = loss_val, batch * len(X)
            print(f"loss: {loss_val}")
        




In [11]:
for iter in np.arange(epochs):
    print(f"Epoch {iter}\n-------------------------------")
    train(train1_dataloader,model,loss_fn,optimizer)
    scheduler.step()

Epoch 0
-------------------------------
loss: 48.6239013671875
Epoch 1
-------------------------------
loss: 13.85482120513916
Epoch 2
-------------------------------
loss: 7.729384422302246
Epoch 3
-------------------------------
loss: 6.316256523132324
Epoch 4
-------------------------------
loss: 5.631008625030518
Epoch 5
-------------------------------
loss: 5.1686224937438965
Epoch 6
-------------------------------
loss: 4.844146251678467
Epoch 7
-------------------------------
loss: 4.6106390953063965
Epoch 8
-------------------------------
loss: 4.528166770935059
Epoch 9
-------------------------------
loss: 4.542829513549805
Epoch 10
-------------------------------
loss: 4.324438571929932
Epoch 11
-------------------------------
loss: 4.200567722320557
Epoch 12
-------------------------------
loss: 4.241685390472412
Epoch 13
-------------------------------
loss: 4.083768367767334
Epoch 14
-------------------------------
loss: 4.108485698699951
Epoch 15
-------------------------

In [12]:
mean, theta, pi = model(count_mtx_raw, size_factor)
print(mean)
mean_value=mean.detach().numpy()
#print(theta)
#print(pi)
#print(mean.shape,train_data.shape)

tensor([[3.3947e+00, 4.9294e+02, 3.6475e+01,  ..., 1.0418e-01, 1.1592e+00,
         4.1442e+01],
        [4.7040e+00, 6.9294e+02, 5.2233e+01,  ..., 6.5395e-02, 1.6272e+00,
         4.7663e+01],
        [3.1404e+00, 4.1121e+02, 3.0450e+01,  ..., 5.7691e-02, 9.4699e-01,
         3.0292e+01],
        ...,
        [4.6547e+00, 4.9638e+02, 4.6179e+01,  ..., 1.0357e-01, 1.5509e+00,
         7.1354e+01],
        [4.4502e+00, 7.4635e+02, 5.0571e+01,  ..., 5.9038e-02, 1.6093e+00,
         5.8876e+01],
        [3.8597e+00, 5.8771e+02, 4.3817e+01,  ..., 1.9420e-01, 1.2540e+00,
         2.0852e+01]], grad_fn=<MulBackward0>)


In [13]:
import copy
def impute_zero(mean, previous_data):
    imputed=previous_data.copy()
    print(imputed)
    for item in np.arange(imputed.shape[0]):
        for ele in np.arange(imputed.shape[1]):
            if imputed[item][ele]==0:
                imputed[item][ele]=mean[item][ele]
    return imputed
    
    

In [14]:
imputed_res=impute_zero(mean,data)
print(imputed_res)


[[  0 542  48 ...   0   0  32]
 [  7 772  63 ...   0   0  62]
 [  0 449  29 ...   0   0  20]
 ...
 [  1 513  24 ...   0   1  38]
 [  0 786  32 ...   0   0  73]
 [  0 640  98 ...   0   3  20]]
[[  3 542  48 ...   0   1  32]
 [  7 772  63 ...   0   1  62]
 [  3 449  29 ...   0   0  20]
 ...
 [  1 513  24 ...   0   1  38]
 [  4 786  32 ...   0   1  73]
 [  3 640  98 ...   0   3  20]]


In [15]:
def previous_diff(previous,file_path):
    true_data=pd.read_csv(file_path)
    true_data=np.array(true_data)
    true_data=true_data.T
    dist=0
    for item in np.arange(previous.shape[0]):
        for ele in np.arange(previous.shape[1]):
            if previous[item][ele]==0:
                res=true_data[item][ele]
                dist=dist+res
    print("Previous Value Sum: ",dist)
def calculate_distance(mat,file_path,imputed):
    distance = 0;
    count_pre,count_after=0,0
    true_data=pd.read_csv(file_path)
    true_data=np.array(true_data)
    true_data=true_data.T
    print(data.shape,true_data.shape)
    for item in np.arange(mat.shape[0]):
        for ele in np.arange(mat.shape[1]):
            if mat[item][ele]==0:
                count_pre+=1
                if np.round(imputed[item][ele])==0:
                    count_after+=1
                res=np.round(imputed[item][ele]-true_data[item][ele])
                #print(mat[item][ele]-true_data[item][ele])
                distance+=np.abs(res)
    print("Dropout Previous :",count_pre/(mat.shape[0]*mat.shape[1]))
    print("Dropout Imputed :",count_after/(mat.shape[0]*mat.shape[1]))
    print("Imputed Value Sum:",distance)
    return distance
dist=calculate_distance(data,"test_truedata.csv",mean_value)
previous_diff(data,"test_truedata.csv")

(200, 1000) (200, 1000)
Dropout Previous : 0.236915
Dropout Imputed : 0.060635
Imputed Value Sum: 99373.0
Previous Value Sum:  157508


对大数据集进行计算

In [16]:
data_test=pd.read_csv("data.csv")
test_data=data_test.T
test_data.to_csv("data4test.csv")
finaldata=test_data.values
data_values=finaldata.T  #进行转置？
data2test=sc.read_csv("data4test.csv")
data2test=data2test[1:data2test.shape[0]]
print(data2test.shape)



(5000, 1000)


In [17]:
sc.pp.filter_cells(data2test, min_counts=1)
data2test.raw = data2test.copy()
data2test.obs['size_factors'] = data2test.obs.n_counts / np.median(data2test.obs.n_counts)
sc.pp.log1p(data2test)
sc.pp.scale(data2test)
count_mtx_raw_1 = torch.tensor(data2test.raw.X)
size_factor_1 = torch.tensor(data2test.obs.size_factors.values)
means, theta, pi = model(count_mtx_raw_1, size_factor_1)
mean_value_final=means.detach().numpy()
print(mean_value_final.shape,test_data.shape)



Trying to set attribute `.obs` of view, copying.


(5000, 1000) (5000, 1000)


In [18]:
results=impute_zero(mean_value_final,finaldata)
final_res=pd.DataFrame(results.T)
final_res.to_csv("results.csv",index=0,header=0)


[[  0   5 236 ...   4  31   6]
 [  0   0 313 ...   8  12   4]
 [  5  13 490 ...  13  22   6]
 ...
 [  3  10 245 ...  12  37   5]
 [  0   5 183 ...   7  14   0]
 [  4   1 481 ...  17  26  20]]


In [19]:
cnt=np.where(results,0,1)
print(np.sum(cnt)/5000000)

0.0240914
